# Advanced Querying Mongo

Importing libraries and setting up connection

In [25]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client['Ironhack']
c = db.get_collection('Companies')
import pandas as pd
import time

In [32]:
db.list_collection_names()

['Companies', 'restaurants']

In [33]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'Companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [49]:
filter_ = {'name':"Babelgum"}
projection = {'name':1, '_id':0}
result = list(c.find(filter_, projection))
result


[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [50]:
filter_ = {'number_of_employees': {'$gt':5000}}
projection = {'name':1, '_id':0, 'number_of_employees':1}
list(c.find(filter_, projection).sort('number_of_employees', -1))[:20]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [70]:
filter_ = {'founded_year': {'$gte' : 2000, '$lte':2005}}
projection = {'name':1, 'founded_year':1, '_id':0}
list(c.find(filter_, projection))[:10]

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [65]:
list(c.find({'$and':
             [{'ipo.valuation_amount':{'$gte': 100000000}},
              {'founded_year':{'$lt': 2010}}]},
        {'_id':0, 'name':1, 'ipo':1}))[:1]

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [74]:
filter_ = {'$and': [{'number_of_employees': { '$lt': 1000 }},{ 'founded_year': { '$lt': 2005 }}]}
projection = {'name':1, '_id':0}
list(c.find(filter_, projection).sort('number_of_employees'))[:10]

[{'name': 'Fox Interactive Media'},
 {'name': 'Skype'},
 {'name': 'Ticketmaster'},
 {'name': 'stylediary'},
 {'name': 'MindTouch'},
 {'name': 'Simpy'},
 {'name': 'Eurekster'},
 {'name': 'Compete'},
 {'name': 'EditGrid'},
 {'name': 'Monster'}]

### 6. All the companies that don't include the `partners` field.

In [85]:
#filter_ = {'partners': {'$exists': False}}
#projection = {'name':1, '_id':0}
list(c.find({'partners': {'$exists': False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [88]:
list(c.find({'category_code': {'$type': 'null'}}))[:1]

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'permalink': 'collective',
  'crunchbase_url': 'http://www.crunchbase.com/company/collective',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'updated_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'overview': None,
  'image': None,
  'products': [],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
  'milestones': [],
  'ipo': N

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [93]:
filter_ = {'number_of_employees': {'$gte': 100, '$lt':1000 }}
projection = {'name':1, 'number_of_employees':1, '_id':0}
list(c.find(filter_, projection))[:10]

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [102]:
#list(c.find().sort('ipo', -1))[:1]
list(c.find({},{'name':1, 'ipo.valuation_amount':1, '_id':0}).sort('ipo.valuation_amount', -1))[:5]

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [104]:
list(c.find({},{'name':1, 'number_of_employees':1, '_id':0}).sort('number_of_employees', -1))[:10]

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [111]:
list(c.find({'founded_month': {'$gt': 6}},{'name':1, '_id':0}).sort('number_of_employees', -1).limit(1000))[:5]

[{'name': 'PayPal'},
 {'name': 'ExxonMobil'},
 {'name': 'Google'},
 {'name': 'kalimatapla'},
 {'name': 'The Walt Disney Company'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [122]:
list(c.find({'$and': [{'founded_year': { '$lt': 2000 }},{'acquisition.price_amount': {'$gt':10000000}}]}, {'name':1, '_id':0}))[:5]

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [127]:
list(c.find({'acquisition.acquired_year': {'$gt': 2010}}, {'name':1, 'acquisition':1, '_id':0}).sort('acquisition.price_amount',-1))[:5]

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [130]:
list(c.find({}, {'name':1, 'founded_year':1, '_id':0}).sort('founded_year', -1))[:5]

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [131]:
list(c.find({"founded_day": {"$lte":7}},{"_id":0, "name":1, "founded_day":1, "acquisition.price_amount":1}).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense',
  'founded_day': 1,
  'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee',
  'founded_day': 1,
  'acquisition': {'price_amount': 321000000}},
 {'name': 'BlueLithium',
  'founded_day': 1,
  'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [132]:
list(c.find({"$and": 
             [{"category_code":"web"}, 
              {"number_of_employees":{"$gt": 4000}}]}, 
        {"_id":0, "name":1, "category_code":1, "number_of_employees":1}).sort("number_of_employees",1))[:5]

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [133]:
list(c.find({"$and": 
             [{"acquisition.price_currency_code":"EUR"}, 
              {"acquisition.price_amount":{"$gt": 10000000}}]}, 
        {"_id":0, "name":1, "acquisition.price_currency_code":1, "acquisition.price_amount":1}))[:5]

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [134]:
list(c.find({"acquisition.acquired_month": {"$lte":3}},{"_id":0, "name":1, "acquisition":1}).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [135]:
list(c.find({"$and": 
             [{"founded_year":{"$gte":2000}},
              {"founded_year":{"$lte":2010}},
              {"acquisition.acquired_year":{"$gte": 2011}}]}, 
        {"_id":0, "name":1, "founded_year":1, "acquisition.acquired_year":1}))[:5]

[{'name': 'Wetpaint',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2013}},
 {'name': 'Digg',
  'founded_year': 2004,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Geni',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2012}},
 {'name': 'Kyte',
  'founded_year': 2006,
  'acquisition': {'acquired_year': 2011}},
 {'name': 'Jingle Networks',
  'founded_year': 2005,
  'acquisition': {'acquired_year': 2011}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [136]:
list(c.find({"$and": 
             [{"founded_year" : {"$type":16}},
              {"deadpooled_year" : {"$type":16}},
              {"$where" : "this.deadpooled_year - this.founded_year > 3"}]}, 
        {"_id":0, "name":1, "founded_year":1, "deadpooled_year":1}).sort("deadpooled_year",1))[:5]

[{'name': 'Excite@Home', 'founded_year': 1995, 'deadpooled_year': 2001},
 {'name': 'RealNames', 'founded_year': 1996, 'deadpooled_year': 2002},
 {'name': 'MediaRush', 'founded_year': 2000, 'deadpooled_year': 2004},
 {'name': 'AllofMP3', 'founded_year': 2000, 'deadpooled_year': 2007},
 {'name': 'FuckedCompany', 'founded_year': 2000, 'deadpooled_year': 2007}]